In [68]:
import sqlite3
# import ibis
import sys
import pandas as pd
sys.path.append("..")
from rcn_py import database
from rcn_py import neo4j_scopus
from neo4j import GraphDatabase
import os


    author-paper table
    author info table
    publication table

In [12]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [12]:
cur.execute("CREATE TABLE test_paper(doi primary key, title, abstract)")

In [13]:
cur.execute("CREATE TABLE test_authors(orcid primary key, name, country)")


In [16]:
cur.execute("CREATE TABLE test_author_papers2(orcid, doi, primary key(orcid, doi) FOREIGN KEY(orcid) REFERENCES test_authors(orcid), FOREIGN KEY(doi) REFERENCES test_paper(doi))")

In [55]:
con.commit()

In [56]:
con.close()

## Insert data

In [72]:
database.insert_database("/Users/jennifer/RCSNA/data/tutorial.db", '',"Peter Kalverla")

'Done.'

In [ ]:
database.insert_coauthors_pub("/Users/jennifer/RCSNA/data/tutorial.db", "Peter Kalverla")

In [ ]:
database.insert_cocoauthors("/Users/jennifer/RCSNA/data/tutorial.db")

### Fetch data from database

In [ ]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

# Topic modeling (title or abstract) to cluster papers
clusters, idx2topics = database.pub_cluster(cur)

In [ ]:
# get coauthor relationship
links = database.fetch_relationships(cur)

In [ ]:
# assign author's group as the most frequent paper group
orcid_group = database.author_cluster(cur, clusters)

cur.close()

In [ ]:
# pyvis
database.build_network_database("Database")

### Store more data

In [4]:
con = sqlite3.connect("/Users/jennifer/RCSNA/data/tutorial.db")
cur = con.cursor()

In [6]:
con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
cur_rsd = con_rsd.cursor()

In [18]:
res_rsd = cur_rsd.execute("""
SELECT orcid, given_name, family_name, affiliation 
FROM author 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
UNION
SELECT orcid, given_name, family_name, affiliation 
FROM contributor 
WHERE affiliation = 'Netherlands eScience Center'
GROUP BY orcid
""")
escience_member = res_rsd.fetchall()

In [25]:
escience_member[0]


('0000-0001-5301-3276',
 'Jesus',
 'Garcia Gonzalez',
 'Netherlands eScience Center')

In [2]:
def insert_escience_rsd_member(db_path):
    con_rsd = sqlite3.connect("/Users/jennifer/RCSNA/data/rsd.db")
    cur_rsd = con_rsd.cursor()
    res_rsd = cur_rsd.execute("""
        SELECT orcid, given_name, family_name, affiliation 
        FROM author 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        UNION
        SELECT orcid, given_name, family_name, affiliation 
        FROM contributor 
        WHERE affiliation = 'Netherlands eScience Center'
        GROUP BY orcid
        """)
    escience_members = res_rsd.fetchall()
    for member in escience_members:
        orcid_id = member[0]
        fullname = member[1] + " " + member[2]
        database.insert_database(db_path, orcid_id, fullname)

    con_rsd.commit()
    con_rsd.close()
    return "Insertion complete"
    

In [3]:
insert_escience_rsd_member("/Users/jennifer/RCSNA/data/tutorial.db")

'Insertion complete'

In [14]:
# cur.execute("DELETE FROM author_publication WHERE doi = 'None'")
con.commit()
con.close()

## Neo4j

In [6]:
def get_pub_info(tx, df, subject):
    for i in range(len(df)):
        doi = df.DOI[i]
        title = df.Title[i]
        year = df.Year[i]
        cited = df["Cited by"][i]
        # subject = subject
        if not isinstance(df['Author Keywords'][i], str):
            keywords = []
        else:
            keywords = df["Author Keywords"][i].split("; ")
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]
        author_aff = df['Authors with affiliations'][i].split("; ")[0:len(author_scopus_id)]
        author_country = [aff.split(", ")[-1] for aff in author_aff]

        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue

        # APOC plugin should be installed in your Neo4j Server
        # Create people nodes
        for n in range(len(author_scopus_id)):
            # if the person exists, append keywords and year
            # avoid adding duplicate years
            tx.run("""
                MERGE (p:Person {scopus_id: $id})
                SET p.name = $name,
                    p.affiliation = $affiliation, 
                    p.country = $country,
                    p.keywords = coalesce(p.keywords, []) + $keywords,
                    p.year = apoc.coll.toSet(coalesce(p.year, []) + $year),
                    p.subject = coalesce(p.subject, []) + $subject
                """, 
                id = author_scopus_id[n],
                name = author_name[n],
                affiliation = author_aff[n],
                country = author_country[n],
                keywords = keywords,
                year = year,
                subject = subject
                )
            
        # Create edges
        for i in range(len(author_scopus_id))[0:-1]:
            for j in range(len(author_scopus_id))[i+1:]:
                # if the edge exists, add weight, doi, title, year
                # avoid adding duplicate years
                tx.run("""
                    MATCH 
                        (a:Person), 
                        (b:Person)
                    WHERE a.scopus_id = $person_a AND b.scopus_id = $person_b
                    MERGE (a)-[r:COAUTHOR]-(b)
                    SET r.doi = coalesce(r.doi, []) + $doi,
                        r.title = coalesce(r.title, []) + $title,
                        r.year = apoc.coll.toSet(a.year + b.year),
                        r.weight = size(r.doi)+1,
                        r.cited = coalesce(r.cited, []) + $cited,
                        r.subject = apoc.coll.toSet(coalesce(r.subject, []) + $subject)
                    """, 
                    person_a=author_scopus_id[i], 
                    person_b=author_scopus_id[j], 
                    doi = doi,
                    title = title,
                    cited = cited,
                    subject = subject
                    )

        

In [116]:
def neo4j_create_people(tx, df, subject):
    for i in range(len(df)):
        # remove noise data
        if not isinstance(df['Author(s) ID'][i], str):
            continue
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]

        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue
        year = df.Year[i]
        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]
        author_aff = df['Authors with affiliations'][i].split("; ")[0:len(author_scopus_id)]
        author_country = [aff.split(", ")[-1] for aff in author_aff]
        if not isinstance(df['Author Keywords'][i], str):
            keywords = []
        else:
            keywords = df["Author Keywords"][i].split("; ")

        # APOC plugin should be installed in your Neo4j Server
        # Create people nodes
        for n in range(len(author_scopus_id)):
            # if the person exists, append keywords and year
            # avoid adding duplicate years
            tx.run("""
                MERGE (p:Person {scopus_id: $id})
                SET p.name = $name,
                    p.affiliation = $affiliation, 
                    p.country = $country,
                    p.keywords = apoc.coll.toSet(coalesce(p.keywords, []) + $keywords),
                    p.year = apoc.coll.toSet(coalesce(p.year, []) + $year),
                    p.subject = apoc.coll.toSet(coalesce(p.subject, []) + $subject)
                """, 
                id = author_scopus_id[n],
                name = author_name[n],
                affiliation = author_aff[n],
                country = author_country[n],
                keywords = keywords,
                year = year,
                subject = subject
                )
        

In [117]:
def neo4j_create_publication(tx, df, subject):
    for i in range(len(df)):
        # remove noise data
        if not isinstance(df['Author(s) ID'][i], str):
            continue
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue

        doi = df.DOI[i]
        title = df.Title[i]
        year = df.Year[i]
        cited = df["Cited by"][i]
        # subject = subject
        if not isinstance(df['Author Keywords'][i], str):
            keywords = []
        else:
            keywords = df["Author Keywords"][i].split("; ")

        # Create publication nodes
        tx.run("""
                MERGE (p:Publication {doi: $doi})
                SET p.title = $title,
                    p.year = $year, 
                    p.cited = $cited,
                    p.keywords = $keywords,
                    p.subject = apoc.coll.toSet(coalesce(p.subject, []) + $subject)
                """, 
                doi = doi,
                title = title,
                year = year,
                cited = cited,
                keywords = keywords,
                subject = subject
                )
        
        

In [118]:
def neo4j_create_author_pub_edge(tx, df):
    for i in range(len(df)):
        # remove noise data
        if not isinstance(df['Author(s) ID'][i], str):
            continue
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue     

        author_name = df["Authors"][i].split(", ")[0:len(author_scopus_id)]   
        year = df.Year[i]
        doi = df.DOI[i]
        title = df.Title[i]
        # APOC plugin should be installed in your Neo4j Server
        # Create edges
        for i in range(len(author_scopus_id)):
            tx.run("""
                    MATCH 
                        (n:Person {scopus_id: $person_id}), 
                        (p:Publication {doi: $doi})
                    MERGE (n)-[r:IS_AUTHOR_OF]->(p)
                    ON CREATE SET 
                        r.year = $year,
                        r.author_name = $author_name,
                        r.title = $title
                    """, 
                    person_id = author_scopus_id[i], 
                    doi = doi, 
                    year = year,
                    author_name = author_name[i],
                    title = title
                    )

In [103]:
def neo4j_create_relationship(tx, df, subject):
    for i in range(len(df)):
        author_scopus_id = df["Author(s) ID"][i].split(";")[0:-1]
        # remove papers with single author
        if len(author_scopus_id) < 2:
            continue        

        doi = df.DOI[i]
        title = df.Title[i]
        cited = df["Cited by"][i]

        # APOC plugin should be installed in your Neo4j Server
        # Create edges
        for i in range(len(author_scopus_id))[0:-1]:
            for j in range(len(author_scopus_id))[i+1:]:
                # if the edge exists, add weight, doi, title, year
                # avoid adding duplicate years
                tx.run("""
                    MATCH 
                        (a:Person {scopus_id: $person_a}), 
                        (b:Person {scopus_id: $person_b})
                    MERGE (a)-[r:COAUTHOR]-(b)
                    SET r.doi = coalesce(r.doi, []) + $doi,
                        r.title = coalesce(r.title, []) + $title,
                        r.year = apoc.coll.toSet(a.year + b.year),
                        r.weight = size(r.doi)+1,
                        r.cited = coalesce(r.cited, []) + $cited,
                        r.subject = apoc.coll.toSet(coalesce(r.subject, []) + $subject)
                    """, 
                    person_a=author_scopus_id[i], 
                    person_b=author_scopus_id[j], 
                    doi = doi,
                    title = title,
                    cited = cited,
                    subject = subject
                    )
        

In [ ]:
# remote AuraDB, only one for free'
# uri = "neo4j+s://a93e1a64.databases.neo4j.io"
# user = "neo4j"
# password = "ifVwf-BGTZt1ltrNgVbh5Q6aGSxQSksQJSxBUbJb1BE"

In [44]:
# local AuraDB
uri = "bolt://localhost:7687"
user = "neo4j"
password = "zhiningbai"

In [46]:
check_verify =  GraphDatabase.driver(uri, auth=(user, password))
check_verify.verify_connectivity()

In [127]:
nl_social2_20 = pd.read_csv("/Users/jennifer/scopus_data/year13/nl_cs1.csv")
nl_social2_20[0:5]

,Authors,Author(s) ID,Title,Year,Cited by,DOI,Link,Affiliations,Authors with affiliations,Author Keywords
0,"Ma H., Van Der Zee R., Nauta B.",56304214300;7007040526;57209315405;,An integrated 80-V class-D power output stage ...,2013,8.0,10.1109/ESSCIRC.2013.6649079,https://www.scopus.com/inward/record.uri?eid=2...,"IC Design Group, CTIT Research Institute, Univ...","Ma, H., IC Design Group, CTIT Research Institu...",NaN
1,"Bouwman H., Cerpa N.",7004140817;6603334728;,Special issue on cloud computing and electroni...,2013,NaN,10.4067/S0718-18762013000300001,https://www.scopus.com/inward/record.uri?eid=2...,"Delft University of Technology, Netherlands; U...","Bouwman, H., Delft University of Technology, N...",NaN
2,"Jiang J., Makinwa K.A.A., Kindt W.J.",55976393500;6603240637;6602378329;,A continuous-time ripple reduction technique f...,2013,1.0,10.1109/ESSCIRC.2013.6649111,https://www.scopus.com/inward/record.uri?eid=2...,"Electronic Instrumentation Laboratory, DIMES, ...","Jiang, J., Electronic Instrumentation Laborato...",NaN
3,"Driemel A., Har-Peled S.",36184974300;7005460407;,Jaywalking your dog: Computing the fréchet dis...,2013,49.0,10.1137/120865112,https://www.scopus.com/inward/record.uri?eid=2...,Department of Information and Computing Scienc...,"Driemel, A., Department of Information and Com...",Approximation algorithms; Fréchet distance; Ge...
4,Abate A.,56819871700;,Approximation metrics based on probabilistic b...,2013,28.0,10.1016/j.entcs.2013.12.002,https://www.scopus.com/inward/record.uri?eid=2...,"Delft Center for Systems and Control, Technisc...","Abate, A., Delft Center for Systems and Contro...",Formal Approximations; Lyapunov Theory; Markov...


In [113]:
for i in range(len(nl_social2_20)):
    if not isinstance(nl_social2_20['Author(s) ID'][i], str):
        print(i)
    

1090


In [115]:
nl_social2_20.iloc[1090]

Authors                                                          The Publisher
Author(s) ID                                                               NaN
Title                        Correction to: Thematic issue: Multilingualism...
Year                                                                      2020
Cited by                                                                   NaN
DOI                                                 10.1007/s10993-019-09535-y
Link                         https://www.scopus.com/inward/record.uri?eid=2...
Affiliations                                            Dordrecht, Netherlands
Authors with affiliations                                        The Publisher
Author Keywords                                                            NaN
Name: 1090, dtype: object

In [ ]:
# Add constraint
"""
CREATE CONSTRAINT pub_doi IF NOT EXISTS
FOR (p:Publication) REQUIRE p.doi IS UNIQUE

CREATE CONSTRAINT scopus_id IF NOT EXISTS
FOR (n:Person) REQUIRE n.scopus_id IS UNIQUE
"""

In [39]:
# Execution
# Filepath and subject depends on the developer's data
for i in range(1,12):
    path = "/Users/jennifer/scopus_data/year22/nl_medicine"+str(i)+".csv"
    df = pd.read_csv(path)
    subject = "Medicine"

    with GraphDatabase.driver(uri, auth=(user, password)) as driver:
        driver.verify_connectivity()
        with driver.session(database="neo4j") as session:
            # Create nodes & edges
            session.execute_write(neo4j_create_people, df, subject) 
            session.execute_write(neo4j_create_publication, df, subject)
            session.execute_write(neo4j_create_author_pub_edge, df)
    print ("nl_medicine"+str(i)+".csv is Done!")      
            
            

nl_medicine1.csv is Done!
nl_medicine2.csv is Done!
nl_medicine3.csv is Done!
nl_medicine4.csv is Done!
nl_medicine5.csv is Done!
nl_medicine6.csv is Done!
nl_medicine7.csv is Done!
nl_medicine8.csv is Done!
nl_medicine9.csv is Done!
nl_medicine10.csv is Done!
nl_medicine11.csv is Done!


In [76]:
file_subject = [("medicine", "Medicine"),
                ("bioc", "Biochemistry, Genetics and Molecular Biology"),
                ("social", "Social Sciences"),
                ("engineer", "Engineering"),
                ("physics", "Physics and Astronomy"),
                ("cs", "Computer Science"),
                ("env", "Environmental Science"),
                ("agricultural", "Agricultural and Biological Sciences"),
                ("earth", "Earth and Planetary Sciences"),
                ("chemistry", "Chemistry"),
                ("psychology", "Psychology"),
                ("neuroscience", "Neuroscience"),
                ("math", "Mathematics"),
                ("immunology", "Immunology and Microbiology"),
                ("materials", "Materials Science"),
                ("multi", "Multidisciplinary"),
                ("arts", "Arts and Humanities"),
                ("chemicalEngineering", "Chemical Engineering"),
                ("pharmacology", "Pharmacology, Toxicology and Pharmaceutics"),
                ("business", "Business, Management and Accounting"),
                ("energy", "Energy"),
                ("nursing", "Nursing"),
                ("eco", "Economics, Econometrics and Finance"),
                ("health", "Health Professions"),
                ("decision", "Decision Sciences"),
                ("veterinary", "Veterinary"),
                ("dentistry", "Dentistry")
                ]

In [122]:
file_subject[17:18]

[('chemicalEngineering', 'Chemical Engineering')]

In [128]:
# Execution
# 22, 21, 20, 19

with GraphDatabase.driver(uri, auth=(user, password)) as driver:
    driver.verify_connectivity()
    with driver.session(database="neo4j") as session:
        # Create nodes & edges
        # Traversing csv files
        for subject_pair in file_subject:
            for n in range(1, 13):
                filename = "nl_" + subject_pair[0] + str(n)
                path = "/Users/jennifer/scopus_data/year18/"+filename+".csv"
                if os.path.exists(path):
                    df = pd.read_csv(path)
                    subject = subject_pair[1]
            
                    session.execute_write(neo4j_create_people, df, subject) 
                    session.execute_write(neo4j_create_publication, df, subject)
                    session.execute_write(neo4j_create_author_pub_edge, df)
                    print (subject+str(n)+" is Done!")  
                else:
                    if n == 1:
                        print("Cannot find file: "+filename)
                    break    
            

Medicine1 is Done!
Medicine2 is Done!
Medicine3 is Done!
Medicine4 is Done!
Medicine5 is Done!
Medicine6 is Done!
Medicine7 is Done!
Medicine8 is Done!
Medicine9 is Done!
Biochemistry, Genetics and Molecular Biology1 is Done!
Biochemistry, Genetics and Molecular Biology2 is Done!
Biochemistry, Genetics and Molecular Biology3 is Done!
Biochemistry, Genetics and Molecular Biology4 is Done!
Social Sciences1 is Done!
Social Sciences2 is Done!
Social Sciences3 is Done!
Engineering1 is Done!
Engineering2 is Done!
Physics and Astronomy1 is Done!
Physics and Astronomy2 is Done!
Physics and Astronomy3 is Done!
Computer Science1 is Done!
Computer Science2 is Done!
Environmental Science1 is Done!
Environmental Science2 is Done!
Agricultural and Biological Sciences1 is Done!
Agricultural and Biological Sciences2 is Done!
Earth and Planetary Sciences1 is Done!
Earth and Planetary Sciences2 is Done!
Chemistry1 is Done!
Chemistry2 is Done!
Psychology1 is Done!
Psychology2 is Done!
Neuroscience1 is Do

In [42]:
session.close()
driver.close()